In [3]:
from initNotebook import initNotebook
initNotebook()

from config.rootPath import getRootPath
from datasets import load_dataset
import pandas as pd

In [4]:
ds = load_dataset("Intuit-GenSRF/hackathon-somos-nlp-2023-suicide-comments-es-en")

c:\Users\adijo\.conda\envs\psyche\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adijo\.cache\huggingface\hub\datasets--Intuit-GenSRF--hackathon-somos-nlp-2023-suicide-comments-es-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 8824/8824 [00:00<00:00, 503640

In [5]:
train_df = ds["train"].to_pandas()

In [8]:
train_df["labels"].head()

0    []
1    []
2    []
3    []
4    []
Name: labels, dtype: object

In [16]:
# First, explode the labels list since each row can have multiple labels
exploded_df = train_df.explode('labels')

# Create one-hot encoding for the labels
one_hot = pd.get_dummies(exploded_df['labels'], prefix='label')

# Group by index and aggregate with max to get back to original shape
one_hot = one_hot.groupby(level=0).max()

# Join the one-hot encoded columns back to original dataframe
final_df = train_df.join(one_hot)

# Show the first few rows to verify
print(final_df.info())
final_df.drop(columns=["text", "labels", "processed_text", "num_tokens"], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8824 entries, 0 to 8823
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            8824 non-null   object
 1   labels          8824 non-null   object
 2   processed_text  8824 non-null   object
 3   num_tokens      8824 non-null   int64 
 4   text_en         8824 non-null   object
 5   label_selfharm  8824 non-null   bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 353.4+ KB
None


In [17]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8824 entries, 0 to 8823
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text_en         8824 non-null   object
 1   label_selfharm  8824 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 77.7+ KB


In [19]:
# Convert and save DataFrame to CSV with new headers
df_transformed = pd.DataFrame({
    'title': [''] * len(final_df),  # Empty strings for title column
    'usertext': final_df['text_en'],  # Copy text_en to usertext
    'y': final_df['label_selfharm'].map({True: 1, False: 0})  # Map boolean to 1/0
})

# Save to CSV file
output_path = getRootPath().joinpath('data/thoughts/kaggle_data.csv')
df_transformed.to_csv(output_path, index=False)